In [17]:
import re
import subprocess

## Pylint messages

In [18]:
print(subprocess.check_output(['pylint', '--version']).decode())

pylint 1.7.1, 
astroid 1.5.3
Python 3.6.2 | packaged by conda-forge | (default, Jul 23 2017, 22:59:30) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]



In [19]:
output = subprocess.check_output(
    "pylint --list-msgs | grep '^:' | awk '{print $2}' | cut -c 2-6", 
    shell=True, universal_newlines=True)
pylint_messages = output.split()

In [20]:
len(pylint_messages)

221

## Ignored (.pylinrc) messages

In [21]:
import configparser
config = configparser.ConfigParser()
config.read('../pyta/python_ta/.pylintrc')
pylintrc_messages = [m.strip(",") for m in config['MESSAGES CONTROL']['disable'].split()]

In [22]:
len(pylintrc_messages)

121

## Undocumented messages

In [33]:
undocumented_messages = {
    'F0001',
    'I0001',
    'F0010',
    'F0002',
    'F0202',
}

## Covered messages

In [23]:
LINK_RE = re.compile('###.*\{#([A-Z][0-9]+)\}.*')  # looks for '{#E0601}'
HEADER_RE = re.compile('###.*\(([A-Z][0-9]+)\).*')  # looks for '### E0601'

In [24]:
link_messages = []
header_messages = []

with open('../website/index.md') as ifh:
    for line in ifh:
        link_messages += LINK_RE.findall(line)
        header_messages += HEADER_RE.findall(line)

In [25]:
assert link_messages == header_messages

In [26]:
covered_messages = link_messages

In [27]:
len(covered_messages)

93

## Missing messages

In [35]:
# Messages in index.md but not in pylint
set(covered_messages) - set(pylint_messages)

{'E0002', 'E9991', 'E9996', 'E9998', 'E9999', 'R0101', 'R0102'}

In [36]:
# Messages in index.md and in .pylintrc
set(covered_messages) & set(pylintrc_messages)

{'E0202', 'E0241', 'E0704', 'W0211'}

In [37]:
# Messages produced by pylint that are not covered by .pylintrc or index.md
missing_messages = set(pylint_messages) - set(pylintrc_messages) - set(covered_messages) - undocumented_messages

In [38]:
len(missing_messages)

25

In [39]:
missing_message_text = {}

for code in missing_messages:
    print(code)
    message = subprocess.check_output("pylint --help-msg={}".format(code), universal_newlines=True, shell=True)
    missing_message_text[code] = message.strip() 

C0201
C0325
R0123
E0303
R1707
C0326
W0312
E1137
W0223
W0106
R1704
R1702
W0311
W0221
C0321
W0301
C0304
C0305
C0330
E0107
E0118
E1133
E1123
C0301
E1138


In [ ]:
RE_MESSAGE_NAME = re.compile(":([a-zA-Z\-]+) \(([A-Z0-9]+)\):")

In [51]:
new_errors_text = """
## New errors {#new}

"""

for code, text in missing_message_text.items():
    match = RE_MESSAGE_NAME.findall(text)
    assert len(match) == 1
    name, code_ = match[0]
    assert code == code_
    text = """\
### {1} ({0}) {{#{0}}}

{3}

~~~~ {{include="{0}_{2}"}}
~~~~

""".format(code, name.replace('-', ' ').capitalize(), name.replace('-', '_'), text)
    new_errors_text += text

print(new_errors_text)


## New errors {#new}

### Consider iterating dictionary (C0201) {#C0201}

:consider-iterating-dictionary (C0201): *Consider iterating the dictionary directly instead of calling .keys()*
  Emitted when the keys of a dictionary are iterated through the .keys() method.
  It is enough to just iterate through the dictionary itself, as in "for key in
  dictionary". This message belongs to the refactoring checker.

~~~~ {include="C0201_consider_iterating_dictionary"}
~~~~

### Superfluous parens (C0325) {#C0325}

:superfluous-parens (C0325): *Unnecessary parens after %r keyword*
  Used when a single item in parentheses follows an if, for, or other keyword.
  This message belongs to the format checker.

~~~~ {include="C0325_superfluous_parens"}
~~~~

### Literal comparison (R0123) {#R0123}

:literal-comparison (R0123): *Comparison to literal*
  Used when comparing an object to a literal, which is usually what you do not
  want to do, since you can compare to a different literal than what was


In [43]:
RE_MESSAGE_NAME.findall(":superfluous-parens (C0325): *Unnecessary parens after %r keyword*")

[('superfluous-parens', 'C0325')]